In [1]:
# Authenticate to project
if [[ "${OS_PROJECT_NAME:+x}" != "x" ]]; then
  echo "No project could automatically be detected. Overriding in Notebook."
  # For example...
  export OS_PROJECT_NAME='Chameleon'
fi

# Test that you can authenticate
openstack token issue >/dev/null && echo "Successfully authenticated to project $OS_PROJECT_NAME"

No project could automatically be detected. Overriding in Notebook.
Successfully authenticated to project Chameleon


In [2]:
# Server info
LEASE_NAME="$USER-chibox"
SERVER_NAME="$LEASE_NAME"
KEYPAIR_NAME="$USER-jupyter-$(hostname)"
# FLAVOR="m1.large"
FLAVOR="baremetal"
NODE_TYPE="compute_haswell"
IMAGE_NAME="CC-CentOS8"

#lease for 1 day
START_DATE="$(date +'%Y-%m-%d %H:%M')"
END_DATE="$(date +'%Y-%m-%d %H:%M' -d'+1 day')"

# Get the ID for the network named "public"
EXT_NETWORK_ID="$(openstack network show public -f value -c id)"

# Use sharednet for public net
PUB_NETWORK_NAME="sharednet1"
PUB_NETWORK_ID="$(openstack network show -f json $PUB_NETWORK_NAME | jq -r '.id')"
PUB_SUBNET_ID="$(openstack network show -f json $PUB_NETWORK_NAME | jq -r '.subnets[]')"

# Set up port on public network with two fixed-IPs and two floating IPs
PUB_PORT_NAME="$LEASE_NAME-pub-if"
FIP_LEASE_NAME="$LEASE_NAME-pub-ip"
FIP_COUNT=2

# vlan for private net
PRIV_NETWORK_NAME="$LEASE_NAME-priv-net"

In [3]:
#Get lease for the server, floatingIPs, and private vlan
blazar lease-show -f value -c status "$LEASE_NAME" || \
blazar lease-create \
    --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
    --reservation resource_type="virtual:floatingip",network_id="$EXT_NETWORK_ID",amount="$FIP_COUNT" \
    --reservation resource_type="network",network_name="${PRIV_NETWORK_NAME}" \
    --end-date "${END_DATE}" \
    "$LEASE_NAME"

ACTIVE


In [4]:
wait_lease "$LEASE_NAME"

Waiting up to 300 seconds for lease shermanm-chibox to start...
Lease started successfully!


In [5]:
# Create the port if it doesn't exist
if [[ $(openstack port show ${PUB_PORT_NAME} ) ]]
then
    PUB_PORT_JSON="$(openstack port show -f json "${PUB_PORT_NAME}" | jq -r '.')"
    PUB_PORT_ID="$( echo ${PUB_PORT_JSON} | jq -r '.id' )"
    echo "port id is: $PUB_PORT_ID"
else
    PUB_PORT_JSON="$(openstack port create -f json \
        --network ${PUB_NETWORK_NAME} \
        --fixed-ip subnet=${PUB_SUBNET_ID} \
        --fixed-ip subnet=${PUB_SUBNET_ID} \
        ${PUB_PORT_NAME})"
fi

PUB_PORT_IPS="$( echo ${PUB_PORT_JSON} | jq -r '.fixed_ips[] | .ip_address')"
echo $PUB_PORT_IPS

port id is: 377948b0-adb2-4307-b4e3-f71edbe5794c
10.140.83.183 10.140.83.218


In [6]:
# TODO, something broken in the logic here
# #Get floating IPs associated with the lease
# FIP_LEASE_ID="$(blazar lease-show -f json -c reservations "$LEASE_NAME" \
#     | jq -r ".reservations" \
#     | jq -r '. | select(.resource_type == "virtual:floatingip").id')"
    
# readarray -t FLOATING_IP_LIST < <(openstack floating ip list -f value \
#     -c "Floating IP Address" \
#     --tags "reservation:$FIP_LEASE_ID")

# #Attach floating IPs to the port
# for i in ${!PUB_PORT_IPS[@]}; do
#     openstack floating ip set  --port "${PUB_PORT_NAME}"  --fixed-ip-address ${PUB_PORT_IPS[$i]}  "${FLOATING_IP_LIST[$i]}"
# done

In [7]:
openstack keypair show "$KEYPAIR_NAME" || \
openstack keypair create --public-key ~/work/.ssh/id_rsa.pub "$KEYPAIR_NAME"

+-------------+------------------------------------------------------------------+
| Field       | Value                                                            |
+-------------+------------------------------------------------------------------+
| created_at  | 2021-02-16T17:33:57.000000                                       |
| deleted     | False                                                            |
| deleted_at  | None                                                             |
| fingerprint | 98:71:e6:98:52:66:0b:ac:03:f1:76:5d:02:f3:6a:89                  |
| id          | 6286                                                             |
| name        | shermanm-jupyter-9d610d205298                                    |
| updated_at  | None                                                             |
| user_id     | d34b62393cdbd07a628635f223217d27640b8a074d04690bbcc6631151539fee |
+-------------+------------------------------------------------------------------+


In [9]:
RESERVATION_ID="$(blazar lease-show -f json -c reservations "$LEASE_NAME" \
     | jq -r ".reservations" \
     | jq -r '. | select(.resource_type == "physical:host").id')"

openstack server show $SERVER_NAME || \
openstack server create \
    --flavor "$FLAVOR" \
    --key-name "$KEYPAIR_NAME" \
    --image "$IMAGE_NAME" \
    --port "$PUB_PORT_NAME" \
    --hint reservation="$RESERVATION_ID" \
    $SERVER_NAME

+-------------------------------------+------------------------------------------------------------------+
| Field                               | Value                                                            |
+-------------------------------------+------------------------------------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                                           |
| OS-EXT-AZ:availability_zone         | nova                                                             |
| OS-EXT-SRV-ATTR:host                | admin01-ironic                                                   |
| OS-EXT-SRV-ATTR:hypervisor_hostname | 224e4adf-e622-4d1d-8981-01082867b561                             |
| OS-EXT-SRV-ATTR:instance_name       | instance-0000c746                                                |
| OS-EXT-STS:power_state              | Running                                                          |
| OS-EXT-STS:task_state              

In [10]:
# openstack server show "$SERVER_NAME"
wait_instance "$SERVER_NAME"

Waiting up to 600 seconds for instance shermanm-chibox to start
Instance created successfully!


In [12]:
#the first IP is unassigned. The second is used for ssh.
FLOATING_IP="192.5.87.223"
wait_ssh $FLOATING_IP

Waiting up to 300 seconds for SSH on 192.5.87.223...
SSH is running!


In [13]:
ssh cc@"$FLOATING_IP" echo 'User $(whoami) connected on $(hostname)!'

User cc connected on shermanm-chibox.novalocal!


In [ ]:
#TODO: Print network subnets and IPs to use for chi-in-a-box

In [ ]:
#TODO: add subnet and port to private network

## Step 5: Clean up (optional)

If you don't need your server anymore, it is a nice thing to end your lease yourself, rather than waiting for it to expire. Ending a lease immediately returns resources to be reserved by others. You don't need to stop your servers; that will be done automatically when the lease is deleted.

In [ ]:
# Commented out for safety.
# blazar lease-delete "$LEASE_NAME"